In [2]:
from __future__ import print_function
import copy

# Import modules required 
import sys, time, os, ast, pickle, copy
from rdkit import Chem
from rdkit.Chem import AllChem
from glob import glob
# from edge_fingerprint import generate_fp_oddt, generate_fp, atom_featurize, get_properties
import numpy as np
from IPython.lib.pretty import pretty
from itertools import chain

In [3]:
    do1() # Generate Our SDF data
    do2() # read dump file
    do3() # Generate Zaretzki's data
    do4() # read dump file
    do5() # Generate ATF map template
    do6() # Generate curated data
    do7() # read dump file
    do8() # Generate training data

NameError: name 'do1' is not defined

In [245]:
fn="./SDF/*.sdf"
def connect_search(filename):
    n_human,not_human=0,0
    supplier = Chem.SDMolSupplier(str(filename), sanitize=False,strictParsing=False)
    
    smiles=[]
    printable=[]
    for mol in supplier:
        idx2mol[int(mol.GetProp('_Name'))] = Chem.RemoveHs(mol)
        
        smiles.append(Chem.MolToSmiles(mol)) ## for making SMARTS (transformer)
        
    ids=list(idx2mol.keys()) ; ids.sort()
    connec_pairs = supplier[0].GetProp('Connectivity')
    connec_pairs = ast.literal_eval(connec_pairs) # Trans type ; str to list
            
#     print("filename : ",filename)
    
    metabolism=[]
    for connec_pair in connec_pairs:
        p = connec_pair[0] # Parent
        m_pair = connec_pair[1:] # Metabolite
        pmol = idx2mol[p]
        pmol = Chem.rdmolops.AddHs(pmol)
        patoms = [patom for patom in pmol.GetAtoms()]
        
        for m in m_pair:
            mmol = idx2mol[m]
            mmol_prop = mmol.GetPropsAsDict() #; print(mmol_prop['Metabolism Type'])
            metabolism.append(mmol_prop['Metabolism Type'])
            mmol_prop_som = mmol.GetProp('SOM')
    
            if mmol_prop['Species'] == 'Human': 
                n_human+=1
            else:
                not_human+=1
                
                somlist = ast.literal_eval(mmol_prop['SOM'])
                
                ## 2개 이상 SOM 넣으면 에러나는 문제가 있어서 2개로 고정했다고 함.
#                 if len(somlist) > 2:
#                     somelist = somelist[:2]

            for atom in pmol.GetAtoms():
                atomidx = atom.GetIdx()
#                 if atomidx in somlist:
#                     y = 'SoM'
#                 else:
#                     y = 'NoSoM'

        
#         print(metabolism)
        for i in metabolism:
#             print("before",i)
            i=i.split(",")
#             print("after",i)
            for item in i:
                printable.append(item)
                
#         print("printable",printable)
        return printable,n_human,not_human
                    

In [246]:
printable=[]
human,nohuman=0,0
for filename in glob(fn):
    a,b,c = connect_search(filename)
    human+=b
    nohuman+=c
    for item in a:
        printable.append(item)
# printable

In [248]:
# print(set(printable))
print("The number of metabolite mechanism : ",len(set(printable)))
# set(printable)
print("Human species : ",human)
print("Not Human species : ",nohuman)

The number of metabolite mechanism :  77
Human species :  3820
Not Human species :  460


In [9]:

count=0
smiles=[]
for fn in glob('./CYP_DB_Zaretzki_XenoSite/*.sdf'):
#     print("="*60);print("="*60);print("="*60)
#     print(fn)
    for mol in Chem.SDMolSupplier(fn):
        smiles.append(Chem.MolToSmiles(mol))
        mprop=mol.GetPropsAsDict()
        count+=1
#         print(mprop)
print("The number of metabolite reaction in Zaretzki : ",count)
print("The number of metabolite reaction in Zaretzki : ",len(smiles))

The number of metabolite reaction in Zaretzki :  2683
The number of metabolite reaction in Zaretzki :  2683


In [156]:
filename='./SDF/S260-3.sdf'
idx2item = {}; idx2mol = {}
supplier = Chem.SDMolSupplier(str(filename),sanitize=False,strictParsing=False)

want=3
mol=supplier[want]
mol_name=mol.GetProp('_Name')
idx2mol[int(mol_name)] = Chem.RemoveHs(mol)

Chem.RemoveHs(mol)

a=[]
for atom in mol.GetAtoms():
    atom.GetSmarts()
    
    
a[0]

RuntimeError: Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 183 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2021.09.5
	BOOST: 1_75


In [104]:
metabolic_steps = 0
for mol in supplier:
    mol_name = mol.GetProp('_Name')
    idx2mol[int(mol_name)] = Chem.RemoveHs(mol)
    metabolic_steps += 1
ids = list(idx2mol.keys())
ids.sort()
ids
connec_pairs = supplier[0].GetProp('Connectivity') #! molla
connec_pairs = ast.literal_eval(connec_pairs) # Trans type: str to list
print(connec_pairs)

for connec_pair in connec_pairs:
    p = connec_pair[0] # Parent
    print("parent : ",p)
    
    m_pair = connec_pair[1:] # Metabolite
    print("parent --> Metabolite: ",m_pair)
    pmol = idx2mol[p]
    pmol = Chem.rdmolops.AddHs(pmol)
    patoms = [patom for patom in pmol.GetAtoms()]
    
connec_pair = connec_pairs[0]
p=connec_pair[0]
m_pair=connec_pair[1:]
pmol=idx2mol[p]
patoms=[patom for patom in pmol.GetAtoms()]
patoms[0]



[[0, 1, 2], [1, 3], [3, 5], [2, 4]]
parent :  0
parent --> Metabolite:  [1, 2]
parent :  1
parent --> Metabolite:  [3]
parent :  3
parent --> Metabolite:  [5]
parent :  2
parent --> Metabolite:  [4]


rdkit.Chem.rdchem._ROAtomSeq

In [114]:
m = m_pair[0]
mmol = idx2mol[m]
mmol_prop = mmol.GetPropsAsDict()